## Implementación de un modelo de machine learning en un punto de conexión de inferencia en tiempo real

https://aws.amazon.com/es/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/

Información general
En este tutorial, aprenderá a implementar un modelo de machine learning (ML) entrenado en un punto de conexión de inferencia en tiempo real mediante Amazon SageMaker Studio.

SageMaker Studio es un entorno de desarrollo integrado (IDE) para ML que proporciona una interfaz de cuaderno de Jupyter totalmente administrada en la que puede realizar tareas de ciclo de vida de ML de extremo a extremo, incluida la implementación de modelos. 
SageMaker ofrece diferentes opciones de inferencia para admitir una amplia gama de casos de uso:

Inferencia en tiempo real de SageMaker para cargas de trabajo con requisitos de baja latencia del orden de milisegundos.
Inferencia sin servidor de SageMaker para cargas de trabajo con patrones de tráfico intermitentes o poco frecuentes.
Inferencia asíncrona de SageMaker para inferencias con grandes tamaños de carga o que requieren largos periodos de procesamiento.
Transformación por lotes de SageMaker para ejecutar predicciones en lotes de datos.
En este tutorial, utilizará la opción Inferencia en tiempo real para implementar un modelo XGBoost de clasificación binaria que ya se ha entrenado en un conjunto de datos sintético de reclamaciones de seguros de automóviles. El conjunto de datos consta de detalles y características extraídas de tablas de reclamaciones y clientes junto con una columna de fraude que indica si una reclamación fue fraudulento o no. El modelo predice la probabilidad de que una reclamación sea fraudulenta. Desempeñará el papel de un ingeniero de machine learning para implementar este modelo y ejecutar inferencias de muestra.

In [3]:
!pip install --upgrade -q scipy

In [4]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
import time
import json
import io
from io import StringIO
import base64
import pprint
import re

from sagemaker.image_uris import retrieve

In [5]:
sess = sagemaker.Session()
write_bucket = sess.default_bucket()
write_prefix = "fraud-detect-demo"

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

Los conjuntos de datos que usa para el entrenamiento existen en un bucket público de S3 llamado `sagemaker-sample-files`

In [6]:
sagemaker_role = sagemaker.get_execution_role()

# S3 locations used for parameterizing the notebook run
read_bucket = "sagemaker-sample-files"
read_prefix = "datasets/tabular/synthetic_automobile_claims" 
model_prefix = "models/xgb-fraud"

data_capture_key = f"{write_prefix}/data-capture"

# S3 location of trained model artifact
model_uri = f"s3://{read_bucket}/{model_prefix}/fraud-det-xgb-model.tar.gz"

# S3 path where data captured at endpoint will be stored
data_capture_uri = f"s3://{write_bucket}/{data_capture_key}"

# S3 location of test data
test_data_uri = f"s3://{read_bucket}/{read_prefix}/test.csv"

En SageMaker, existen varios métodos para implementar un modelo entrenado en un punto de conexión de inferencia en tiempo real: el SDK de SageMaker, AWS SDK: Boto3 y la consola de SageMaker. Para obtener más información, consulte Implementación de modelos para inferencia en la Guía para desarrolladores de Amazon SageMaker. El SDK de SageMaker tiene más abstracciones en comparación con AWS SDK: Boto3, y este último expone una API de nivel inferior para un mayor control sobre la implementación del modelo. En este tutorial, implementará el modelo mediante AWS SDK: Boto3. Hay tres pasos que debe seguir en secuencia para implementar un modelo:

crear un modelo de SageMaker a partir del artefacto del modelo
crear una configuración de punto de conexión para especificar propiedades, incluido el tipo de instancia y el recuento
crear el punto de conexión mediante la configuración del punto de conexión

Para crear un modelo de SageMaker con el artefacto del modelo entrenado almacenado en S3, copie y pegue el siguiente código. El método create_model toma como parámetros el contenedor de Docker que contiene la imagen de entrenamiento (para este modelo, el contenedor XGBoost) y la ubicación de S3 del artefacto del modelo. 

Para crear un modelo de SageMaker con el artefacto del modelo entrenado almacenado en S3, copie y pegue el siguiente código. El método create_model toma como parámetros el contenedor de Docker que contiene la imagen de entrenamiento (para este modelo, el contenedor XGBoost) y la ubicación de S3 del artefacto del modelo. 

In [7]:
# Retrieve the SageMaker managed XGBoost image
training_image = retrieve(framework="xgboost", region=region, version="1.3-1")

# Specify a unique model name that does not exist
model_name = "fraud-detect-xgb"
primary_container = {
                     "Image": training_image,
                     "ModelDataUrl": model_uri
                    }

model_matches = sm_client.list_models(NameContains=model_name)["Models"]
if not model_matches:
    model = sm_client.create_model(ModelName=model_name,
                                   PrimaryContainer=primary_container,
                                   ExecutionRoleArn=sagemaker_role)
else:
    print(f"Model with name {model_name} already exists! Change model name to create new")

Después de crear el modelo de SageMaker, copie y pegue el siguiente código para usar el método Boto3 create_endpoint_config a fin de configurar el punto de conexión. Las entradas principales del método create_endpoint_config son el nombre de la configuración del punto de conexión y la información de la variante, como el tipo de instancia de inferencia y el recuento, el nombre del modelo que se implementará y el tráfico compartido que debe manejar el punto de conexión. Junto con esta configuración, también puede configurar la captura de datos al especificar una DataCaptureConfig. Esta característica le permite configurar el punto de conexión en tiempo real para capturar y almacenar las solicitudes o respuestas en Amazon S3. La captura de datos es uno de los pasos para configurar el monitoreo del modelo y, cuando se combina con las métricas de referencia y los trabajos de monitoreo, lo ayuda a monitorear el rendimiento del modelo al comparar las métricas de datos de prueba con las referencias. Tal monitoreo es útil para programar el reentrenamiento del modelo en función de la desviación del modelo o de los datos y con fines de auditoría. En la configuración actual, tanto la entrada (datos de prueba entrantes) como la salida (predicciones del modelo) se capturan y almacenan en su bucket de S3 predeterminado. 

In [9]:
# Endpoint Config name
endpoint_config_name = f"{model_name}-endpoint-config"

# Endpoint config parameters
production_variant_dict = {
                           "VariantName": "Alltraffic",
                           "ModelName": model_name,
                           "InitialInstanceCount": 1,
                           "InstanceType": "ml.m5.xlarge",
                           "InitialVariantWeight": 1
                          }

# Data capture config parameters
data_capture_config_dict = {
                            "EnableCapture": True,
                            "InitialSamplingPercentage": 100,
                            "DestinationS3Uri": data_capture_uri,
                            "CaptureOptions": [{"CaptureMode" : "Input"}, {"CaptureMode" : "Output"}]
                           }


# Create endpoint config if one with the same name does not exist
endpoint_config_matches = sm_client.list_endpoint_configs(NameContains=endpoint_config_name)["EndpointConfigs"]
if not endpoint_config_matches:
    endpoint_config_response = sm_client.create_endpoint_config(
                                                                EndpointConfigName=endpoint_config_name,
                                                                ProductionVariants=[production_variant_dict],
                                                                DataCaptureConfig=data_capture_config_dict
                                                               )
else:
    print(f"Endpoint config with name {endpoint_config_name} already exists! Change endpoint config name to create new")

Copie y pegue el siguiente código para crear el punto de conexión. El método create_endpoint toma la configuración del punto de conexión como parámetro e implementa el modelo especificado en la configuración del punto de conexión en una instancia de computación. Se tarda unos 6 minutos en implementar el modelo.


In [10]:
endpoint_name = f"{model_name}-endpoint"

endpoint_matches = sm_client.list_endpoints(NameContains=endpoint_name)["Endpoints"]
if not endpoint_matches:
    endpoint_response = sm_client.create_endpoint(
                                                  EndpointName=endpoint_name,
                                                  EndpointConfigName=endpoint_config_name
                                                 )
else:
    print(f"Endpoint with name {endpoint_name} already exists! Change endpoint name to create new")

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Endpoint Status: {status}...")
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
print(f"Endpoint Status: {status}")

Endpoint Status: Creating...
Endpoint Status: Creating...
Endpoint Status: Creating...
Endpoint Status: InService


Después de que el estado del punto de conexión cambie a InService, puede invocar el punto de conexión mediante la API de REST, AWS SDK: Boto3, SageMaker Studio, la AWS CLI o el SDK para Python de SageMaker. En este tutorial, utilizará AWS SDK: Boto3. Antes de llamar a un punto de conexión, es importante que los datos de prueba tengan el formato adecuado para el punto de conexión con serialización y deserialización. La serialización es el proceso de convertir datos sin procesar en un formato como .csv en secuencias de bytes que el punto de conexión puede usar. La deserialización es el proceso inverso de convertir la secuencia de bytes en un formato legible para humanos. En este tutorial, invocará el punto de conexión mediante el envío de las primeras cinco muestras desde un conjunto de datos de prueba. Para invocar el punto de conexión y obtener resultados de predicción, copie y pegue el siguiente código. Dado que la solicitud al punto de conexión (conjunto de datos de prueba) está en formato .csv, se usa un proceso de serialización csv para crear la carga. Luego, la respuesta se deserializa en una matriz de predicciones. Una vez finalizada la ejecución, la celda devuelve las predicciones del modelo y las etiquetas verdaderas para las muestras de prueba. Tenga en cuenta que el modelo XGBoost devuelve probabilidades en lugar de etiquetas de clase reales. El modelo predijo una probabilidad muy baja de que las muestras de prueba sean reclamaciones fraudulentas y las predicciones están alineadas con las etiquetas verdaderas.

In [12]:
# Fetch test data to run predictions with the endpoint
test_df = pd.read_csv(test_data_uri)

# For content type text/csv, payload should be a string with commas separating the values for each feature
# This is the inference request serialization step
# CSV serialization
csv_file = io.StringIO()
test_sample = test_df.drop(["fraud"], axis=1).iloc[:5]
test_sample.to_csv(csv_file, sep=",", header=False, index=False)
payload = csv_file.getvalue()
response = sm_runtime_client.invoke_endpoint(
                                             EndpointName=endpoint_name,
                                             Body=payload,
                                             ContentType="text/csv",
                                             Accept="text/csv"
                                            )

# This is the inference response deserialization step
# This is a bytes object
result = response["Body"].read()
# Decoding bytes to a string
result = result.decode("utf-8")
# Converting to list of predictions
result = re.split(",|\n",result)

prediction_df = pd.DataFrame()
prediction_df["Prediction"] = result[:5]
prediction_df["Label"] = test_df["fraud"].iloc[:5].values
prediction_df

,Prediction,Label
0,0.02243666537106037,0
1,0.02243666537106037,0
2,0.0799826979637146,0
3,0.13931523263454437,0
4,0.03112351894378662,0


Para monitorear las métricas de invocación de puntos de conexión con Amazon CloudWatch, abra la consola de SageMaker. En Inference (Inferencia), seleccione **Endpoints** (Puntos de conexión), **fraud-detect-xgb-endpoint**.

En la página Endpoint details (Detalles del punto de conexión), en Monitor (Monitorear), elija View invocation metrics (Ver métricas de invocación). Inicialmente, es posible que vea solo un punto en el gráfico de métricas. Pero después de varias invocaciones, verá una línea similar a la de la captura de pantalla de muestra.

La página Metrics (Métricas) muestra varias métricas de rendimiento del punto de conexión. Puede elegir diferentes periodos, como más de 1 hora o 3 horas, para visualizar el rendimiento del punto de conexión. Seleccione cualquier métrica para ver su tendencia durante el periodo elegido. En el siguiente paso, elija una de estas métricas para definir las políticas de escalado automático.

Debido a que la captura de datos se estableció en la configuración del punto de conexión, tiene una forma de inspeccionar qué carga se envió al punto de conexión junto con su respuesta. Los datos capturados tardan un tiempo en cargarse por completo en S3. Copie y pegue el siguiente código para comprobar si la captura de datos está completa.

In [13]:
from sagemaker.s3 import S3Downloader
print("Waiting for captures to show up", end="")
for _ in range(90):
    capture_files = sorted(S3Downloader.list(f"{data_capture_uri}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    time.sleep(1)
print()
print(f"Found {len(capture_files)} Data Capture Files:")

Waiting for captures to show up..........................................................................................
Found 1 Data Capture Files:


Los datos capturados se almacenan como un archivo separado para cada invocación de punto de conexión en S3 en líneas JSON, un formato delimitado por saltos de línea para almacenar datos estructurados donde cada línea es un valor JSON. Copie y pegue el siguiente código para recuperar los archivos de captura de datos. 


In [14]:
capture_files = sorted(S3Downloader.list(f"{data_capture_uri}/{endpoint_name}"))
capture_file = S3Downloader.read_file(capture_files[0]).split("\n")
capture_record = json.loads(capture_file[0])
capture_record

{'captureData': {'endpointInput': {'observedContentType': 'text/csv',
   'mode': 'INPUT',
   'data': 'MiwwLDAsMSw1ODQwMCwxNDI0Ny43NjY4NjY2NzA4NSw3MjY0Ny43NjY4NjY2NzA4NSw3LDI5LDAsMTEsNDEsMTA4LDAsMSw3NTAsMzAwMCwwLDIsMjAxNCwwLDAsMSwwLDAsMSwwLDAsMCwxLDAsMCwwLDAsMSwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMCwxCjIsMCwyLDAsMTE1MDAsMTA2NzUuNjcxMzQ2NzM4MTU4LDIyMTc1LjY3MTM0NjczODE2LDExLDI4LDMsMTYsNTQsMTcxLDAsMSw3NTAsMjc1MCwyLDMsMjAxNSwwLDAsMCwwLDEsMSwwLDAsMCwxLDAsMCwwLDAsMSwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMCwxCjIsMCwwLDEsMTg1MDAsMTAyMDIuMjY2MzUzOTk5Nzc2LDI4NzAyLjI2NjM1Mzk5OTc3LDEsNiw2LDgsNTEsMTk2LDAsMSw3NTAsMzAwMCwwLDEsMjAxMSwwLDAsMCwwLDEsMSwwLDAsMSwwLDAsMCwwLDEsMCwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMSwwCjEsMCwwLDAsMTYzMDAsOTMzOC4zNDgwNjY0MzU3MTgsMjU2MzguMzQ4MDY2NDM1NzIsOSwxLDYsMyw1NywxMjIsMCwxLDc1MCwzMDAwLDEsMSwyMDE1LDAsMCwwLDEsMCwwLDEsMCwwLDAsMSwwLDAsMCwxLDAsMCwxLDAsMSwwLDAsMCwwLDAsMCwxLDAKMiwxLDAsMCwxNDcwMCwyODE0NS45MjQ5OTQxODIyNSw0Mjg0NS45MjQ5OTQxODIyNSwxMCwyMywyLDE1LDU5LDE0MiwwLDEsNzUwLDMwMDAsMiwzLDIwMTksMCwwLD

El siguiente codigo decodifica los datos de los archivos capturados mediante base64. El código recupera las cinco muestras de prueba que se enviaron como carga y sus predicciones. Esta característica es útil para inspeccionar las cargas de los puntos de conexión con las respuestas del modelo y monitorear el rendimiento del modelo.


In [15]:
input_data = capture_record["captureData"]["endpointInput"]["data"]
output_data = capture_record["captureData"]["endpointOutput"]["data"]
input_data_list = base64.b64decode(input_data).decode("utf-8").split("\n")
print(input_data_list)
output_data_list = base64.b64decode(output_data).decode("utf-8").split("\n")
print(output_data_list)

['2,0,0,1,58400,14247.76686667085,72647.76686667085,7,29,0,11,41,108,0,1,750,3000,0,2,2014,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1', '2,0,2,0,11500,10675.671346738158,22175.67134673816,11,28,3,16,54,171,0,1,750,2750,2,3,2015,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1', '2,0,0,1,18500,10202.266353999776,28702.26635399977,1,6,6,8,51,196,0,1,750,3000,0,1,2011,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0', '1,0,0,0,16300,9338.348066435718,25638.34806643572,9,1,6,3,57,122,0,1,750,3000,1,1,2015,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0', '2,1,0,0,14700,28145.92499418225,42845.92499418225,10,23,2,15,59,142,0,1,750,3000,2,3,2019,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0', '']
['0.02243666537106037', '0.02243666537106037', '0.0799826979637146', '0.13931523263454437', '0.03112351894378662', '']


### Configurar el escalado automático para el punto de conexión

Las cargas de trabajo que utilizan puntos de conexión de inferencia en tiempo real suelen tener requisitos de baja latencia. Además, cuando el tráfico aumenta, los puntos de conexión de inferencia en tiempo real pueden experimentar sobrecarga de CPU, alta latencia o tiempos de espera. Por lo tanto, es importante escalar la capacidad para manejar los cambios de tráfico de manera eficiente con baja latencia. El escalado automático de inferencia de SageMaker monitorea sus cargas de trabajo y ajusta de forma dinámica el recuento de instancias para mantener un rendimiento constante y predecible de los puntos de conexión a un bajo costo. Cuando aumenta la carga de trabajo, el escalado automático pone más instancias en línea y, cuando la carga de trabajo disminuye, elimina las instancias innecesarias, lo que lo ayuda a reducir el costo de computación. En este tutorial, utilizará AWS SDK: Boto3 para configurar el escalado automático en su punto de conexión. SageMaker proporciona varios tipos de escalado automático: escalado de seguimiento de objetivos, escalado por pasos, escalado bajo demanda y escalado programado. En este tutorial, utilizará una política de escalado de seguimiento de objetivos, que se activa cuando una métrica de escalado elegida aumenta por encima de un umbral objetivo elegido.

El escalado automático se puede configurar en dos pasos. En primer lugar, configura una política de escalado con detalles del número mínimo, deseado y máximo de instancias por punto de conexión. Copie y pegue el siguiente código para configurar una política de escalado de seguimiento de objetivos. La cantidad máxima especificada de instancias se lanza cuando el tráfico supera los umbrales elegidos, que selecciona en el siguiente paso.

In [16]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)

# SageMaker expects resource id to be provided with the following structure
resource_id = f"endpoint/{endpoint_name}/variant/{resp['ProductionVariants'][0]['VariantName']}"

# Scaling configuration
scaling_config_response = sm_autoscaling_client.register_scalable_target(
                                                          ServiceNamespace="sagemaker",
                                                          ResourceId=resource_id,
                                                          ScalableDimension="sagemaker:variant:DesiredInstanceCount", 
                                                          MinCapacity=1,
                                                          MaxCapacity=2
                                                        )

Copie y pegue el siguiente código para crear la política de escalado. La métrica de escalado elegida es SageMakerVariantInvocationsPerInstance, que es la cantidad promedio de veces por minuto que se invoca cada instancia de inferencia para una variante del modelo. Cuando este número cruza el umbral elegido de 5, se activa el escalado automático.


In [17]:
# Create Scaling Policy
policy_name = f"scaling-policy-{endpoint_name}"
scaling_policy_response = sm_autoscaling_client.put_scaling_policy(
                                                PolicyName=policy_name,
                                                ServiceNamespace="sagemaker",
                                                ResourceId=resource_id,
                                                ScalableDimension="sagemaker:variant:DesiredInstanceCount",
                                                PolicyType="TargetTrackingScaling",
                                                TargetTrackingScalingPolicyConfiguration={
                                                    "TargetValue": 5.0, # Target for avg invocations per minutes
                                                    "PredefinedMetricSpecification": {
                                                        "PredefinedMetricType": "SageMakerVariantInvocationsPerInstance",
                                                    },
                                                    "ScaleInCooldown": 600, # Duration in seconds until scale in
                                                    "ScaleOutCooldown": 60 # Duration in seconds between scale out
                                                }
                                            )

In [18]:
response = sm_autoscaling_client.describe_scaling_policies(ServiceNamespace="sagemaker")

pp = pprint.PrettyPrinter(indent=4, depth=4)
for i in response["ScalingPolicies"]:
    pp.pprint(i["PolicyName"])
    print("")
    if("TargetTrackingScalingPolicyConfiguration" in i):
        pp.pprint(i["TargetTrackingScalingPolicyConfiguration"])

'scaling-policy-fraud-detect-xgb-endpoint'

{   'PredefinedMetricSpecification': {   'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'},
    'ScaleInCooldown': 600,
    'ScaleOutCooldown': 60,
    'TargetValue': 5.0}


El siguiente codigo realizar una prueba de esfuerzo del punto de conexión. El código se ejecuta durante 250 segundos e invoca el punto de conexión de forma repetida mediante el envío de muestras seleccionadas al azar del conjunto de datos de prueba.

In [19]:
request_duration = 250
end_time = time.time() + request_duration
print(f"Endpoint will be tested for {request_duration} seconds")
while time.time() < end_time:
    csv_file = io.StringIO()
    test_sample = test_df.drop(["fraud"], axis=1).iloc[[np.random.randint(0, test_df.shape[0])]]
    test_sample.to_csv(csv_file, sep=",", header=False, index=False)
    payload = csv_file.getvalue()
    response = sm_runtime_client.invoke_endpoint(
                                                 EndpointName=endpoint_name,
                                                 Body=payload,
                                                 ContentType="text/csv"
                                                )

Endpoint will be tested for 250 seconds


Puede monitorear las métricas de los puntos de conexión con Amazon CloudWatch. Para obtener una lista de las métricas de punto de conexión disponibles, incluidas las de invocación, consulte Métricas de invocación de punto de conexión de SageMaker. En la consola de SageMaker, en Inference (Inferencia), elija Endpoints (Puntos de conexión), fraud-detect-xgb-endpoint. En la página Endpoint details (Detalles del punto de conexión), vaya a la sección Monitor (Monitorear) y elija View invocation metrics (Ver métricas de invocación). En la página Metrics (Métricas), seleccione InvocationsPerInstance (esta es una métrica de monitoreo que eligió cuando configuró la política de escalado) e Invocations (Invocaciones) de la lista de métricas, y luego elija la pestaña Graphed metrics (Métricas gráficas).

En la página Graphed metrics (Métricas gráficas), puede inspeccionar visualmente el patrón de tráfico recibido por el punto de conexión y cambiar la granularidad del tiempo, por ejemplo, de los 5 minutos predeterminados a 1 minuto. Es posible que el escalado automático tarde unos minutos en agregar la segunda instancia. Una vez que se agrega la nueva instancia, notará que las invocaciones por instancia son la mitad del total de invocaciones.

Cuando el punto de conexión recibe el aumento de la carga, puede comprobar el estado del punto de conexión al ejecutar el código a continuación. Este código comprueba cuándo el estado del punto de conexión cambia de InService a Updating y realiza un seguimiento de los recuentos de instancias. Después de unos minutos, puede ver que el estado cambia de InService a Updating y nuevamente a InService, pero con un mayor número de instancias.

In [ ]:
# Check the instance counts after the endpoint gets more load
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response["EndpointStatus"]
request_duration = 250
end_time = time.time() + request_duration
print(f"Waiting for Instance count increase for a max of {request_duration} seconds. Please re run this cell in case the count does not change")
while time.time() < end_time:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response["EndpointStatus"]
    instance_count = response["ProductionVariants"][0]["CurrentInstanceCount"]
    print(f"Status: {endpoint_status}")
    print(f"Current Instance count: {instance_count}")
    if (endpoint_status=="InService") and (instance_count>1):
        break
    else:
        time.sleep(15)

## Eliminar los recursos

Una práctica recomendada es eliminar los recursos que ya no se utilizan para no incurrir en cargos no deseados.

Elimine el modelo, la configuración del punto de conexión y el punto de conexión que creó en este tutorial mediante la ejecución del siguiente bloque de código en su cuaderno. Si no elimina el punto de conexión, su cuenta seguirá acumulando cargos por la instancia de computación que se ejecuta en el punto de conexión.

In [ ]:
# Delete model
sm_client.delete_model(ModelName=model_name)

# Delete endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)